In [8]:
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
import numpy as np

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
 #       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
model = Sequential()
model.add(Conv2D(input_shape=(299,299,3), filters=64, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size= (3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size= (3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size= (3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size= (3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))



model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))

model.add(Dense(units=4096, activation="relu"))

model.add(Dense(units=2, activation="softmax"))

In [11]:
model.summary()

In [10]:
from tensorflow.keras.optimizers import Adam 
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
                                rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                      )
training_generator = training_datagen.flow_from_directory(
    "../input/covid-radio/Training",
    target_size = (299,299),
    batch_size = 50, 
    class_mode="categorical"
    
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    "../input/covid-validation/Validation",
    target_size = (299,299),
    batch_size = 40, 
    class_mode="categorical"
    
)

In [13]:
history = model.fit(
    training_generator,
    steps_per_epoch = 100, 
    epochs = 50, 
    validation_data = validation_generator,
    validation_steps = 50,
    verbose=1
)

In [61]:
import matplotlib.pyplot as plt
import cv2
acc = history.history["accuracy"]
valid_acc = history.history["val_accuracy"]
loss = history.history["loss"]
valid_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs, acc, "bo", label="Training Accuracy")
plt.plot(epochs, valid_acc, "--b", label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.figure()

plt.plot(epochs, loss, "ro", label="Training Loss")
plt.plot(epochs, valid_loss, "--r", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.figure()

In [87]:

from keras.preprocessing import image 
import os 

path = os.path.join("../input/test-set/test/covid")
path = os.listdir(path)
count = 0
for i in path:
    test_path = "../input/test-set/test/covid/"+i
    img = image.load_img(test_path, target_size=(299,299))
    #plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    
    
    
    
    if classes[0][0] < classes[0][1]:
        
        print(str(i), "This report says you have COVID-19 POSITIVE")
    else:
         print(str(i), "This is Normal")




In [91]:

from keras.preprocessing import image 
import os 


        


    
path = "../input/test-set/test/covid/COVID-3501.png"
img = image.load_img(path, target_size=(299,299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
plt.imshow(img)
if classes[0][0] > classes[0][1]:
    print(str(classes)+"-This is Normal")
else:
    print(str(classes) +"-This report says you have COVID-19 POSITIVE")


